In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
train=pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")

In [4]:
#変数は匿名化されているらしく、何を意味するものか分からない。
test.head()

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,822e49b95,...,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.528249e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,20884000.0,0.0,0.0,0.0,0.0,0.0,36252000.0,0.0,20000000.0,0.0,...,0.0,0.0,25010000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(test.dtypes)
#テスト、訓練ともに欠損はなし。ただし項目数が4992と多すぎるうえに何を意味してるかわからないため、何とか意味のある項目を絞りたい。
print(test.isnull().sum()[test.isnull().sum()!=0])
print(train.isnull().sum()[train.isnull().sum()!=0])
#id以外はobject型ではない
print(train.dtypes[train.dtypes=="object"])

ID            object
48df886f9    float64
0deb4b6a8    float64
34b15f335    float64
a8cb14b00    float64
2f0771a37    float64
30347e683    float64
d08d1fbe3    float64
6ee66e115    float64
20aa07010    float64
dc5a8f1d8    float64
11d86fa6a    float64
77c9823f2    float64
8d6c2a0b2    float64
4681de4fd    float64
adf119b9a    float64
cff75dd09    float64
96f83a237    float64
b8a716ebf    float64
6c7a4567c    float64
4fcfd2b4d    float64
f3b9c0b95    float64
71cebf11c    float64
d966ac62c    float64
68b647452    float64
c88d108c9    float64
ff7b471cd    float64
d5308d8bc    float64
0d866c3d7    float64
bc3f77679    float64
              ...   
cdfc2b069    float64
2a879b4f7    float64
6b119d8ce    float64
98dea9e42    float64
9f2471031    float64
88458cb21    float64
f40da20f4    float64
7ad6b38bd    float64
c901e7df1    float64
8f55955dc    float64
85dcc913d    float64
5ca0b9b0c    float64
eab8abf7a    float64
8d8bffbae    float64
2a1f6c7f9    float64
9437d8b64    float64
5831f4c76    

In [ ]:
#相関係数行列の作成
cor_table=train.corr()

In [ ]:
#相関行列にすべてNaNの列がある→その項目は全データ同じものなので、落とすことにする
drop_list=[]
for i in cor_table.columns:
    if len(cor_table[cor_table[i].notnull()])==0:
        drop_list.append(i)

train.drop(drop_list,axis=1,inplace=True)
test.drop(drop_list,axis=1,inplace=True)

In [ ]:
print(len(train.columns))
print(len(test.columns))

In [ ]:
#相関係数が一定以上の変数を削除する関数
#今回は変数間で相関係数が0.7以上の変数の組み合わせのうち、被説明変数のtargetとの相関係数が低いほうの項目を削除する

over_list=[]
small=[]
value=0.7

for j in cor_table.columns:
    overs=list(cor_table[cor_table[j]>value].index)
    for over in overs:
        if [j,over] not in over_list and [over,j] not in over_list and j!=over:
            over_list.append([j,over])
            if cor_table.loc[j,"target"] <= cor_table.loc[over,"target"]:
                small.append(j)
            else:
                small.append(over)
print(small)

In [ ]:
#listの重複を消す
small=list(set(small))

#上で作った変数を削除する
train.drop(small,axis=1,inplace=True)
test.drop(small,axis=1,inplace=True)

In [ ]:
print(len(train.columns))
print(len(test.columns))

In [ ]:
#まだまだ項目が多いので、randomforestで影響が大きいものに絞る
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

train_x=train.drop(["ID","target"],axis=1)
train_y=train["target"]
selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold="3*median")
selector.fit(train_x, train_y)
x_selected = selector.transform(train_x)
test_selected=selector.transform(test.drop("ID",axis=1))

In [ ]:
#項目数は1244まで絞り込まれた
print(x_selected.shape)
print(test_selected.shape)

In [ ]:
#ランダムフォレスト回帰
from sklearn import grid_search

forest=RandomForestRegressor(random_state=2)

# 試行するパラメータを羅列する
params = {'n_estimators'  : [100],'n_jobs': [-1]}

rf = grid_search.GridSearchCV(
    forest,
    param_grid=params,  
    cv=3 
    )

# グリッドサーチで優れたハイパーパラメータを探す
rf.fit(x_selected, train_y.ravel())
    
result=rf.predict(test_selected)

#  提出する
submission = pd.DataFrame({
        "Id": test["ID"],
        "target": result
    })

#  csv形式のファイルに出力
submission.to_csv("submission_rf.csv", index=False)

In [ ]:
#クロスバリデーションの結果
from sklearn import cross_validation
scores=cross_validation.cross_val_score(rf,x_selected,train_y.ravel(),cv=10)
print("交差検証:n=10",np.mean(scores))